## 輸入使用者資料

In [9]:
account = "112024519" # student ID
password = "*****" # password
homework_name = "HW5" # 作業名稱
class_code = "11220STAT216000" #課程代碼
grade_direction = r"C:\Users\STAT-PC\Downloads\成績登記 - 工作表1 (10).csv" #會取csv表的第一張sheet。成績登記表本機位置
directory_path = r"C:\Users\STAT-PC\Downloads\all_done-20240408T003734Z-001\all_done" # 改完作業檔案夾位置

In [2]:
import os

def find_files_with_id(directory, id_str):
    # 創建一個空列表來儲存匹配的文件
    matching_files = []

    # 使用os.walk來遍歷給定目錄下的所有子目錄和文件
    for root, dirs, files in os.walk(directory):
        # 在當前目錄下遍歷所有文件
        for file in files:
            # 如果文件名中包含了給定的id_str字符串
            if id_str in file:
                # 將符合條件的文件的完整路徑加入到列表中
                matching_files.append(os.path.join(root, file))

    # 返回匹配的文件列表
    return matching_files



In [3]:
import pandas as pd
score_df = pd.read_csv(grade_direction)

In [4]:
score_df.head() # 確認讀取正確的成績

,NAME+id,Score
0,0007-109700034 (賴笙瑞),15
1,107000231 (林新為),15
2,108048272 (黃貞馨),19
3,109000127 (郭依雯),0
4,109022521 (岑美茵),18


In [5]:
unique_names = set(score_df["NAME+id"].str.split("(").str[0].str.strip())

In [7]:
def get_value_based_on_id(df, col_1, col_2, search_id):
    """
    根據學號找到學生的成績
    """
    # Filtering the dataframe for rows where col_1 contains the search_id
    result = df[df[col_1].astype(str).str.contains(str(search_id))][col_2]
    
    return result.tolist()

In [ ]:
# 利用JS模仿人類上傳檔案的行為

JS_DROP_FILE = """
    var target = arguments[0],
        offsetX = arguments[1],
        offsetY = arguments[2],
        document = target.ownerDocument || document,
        window = document.defaultView || window;

    var input = document.createElement('INPUT');
    input.type = 'file';
    input.onchange = function () {
      var rect = target.getBoundingClientRect(),
          x = rect.left + (offsetX || (rect.width >> 1)),
          y = rect.top + (offsetY || (rect.height >> 1)),
          dataTransfer = { files: this.files };

      ['dragenter', 'dragover', 'drop'].forEach(function (name) {
        var evt = document.createEvent('MouseEvent');
        evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
        evt.dataTransfer = dataTransfer;
        target.dispatchEvent(evt);
      });

      setTimeout(function () { document.body.removeChild(input); }, 25);
    };
    document.body.appendChild(input);
    return input;
"""

def drag_and_drop_file(drop_target, path):
    driver = drop_target.parent
    file_input = driver.execute_script(JS_DROP_FILE, drop_target, 0, 0)
    file_input.send_keys(path)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

# Initialize the Chrome driver

driver = webdriver.Chrome()
driver.set_window_size(1400,1000)

# Navigate to the login page
driver.get("https://oauth.ccxp.nthu.edu.tw/v1.1/authorize.php?response_type=code&client_id=eeclass&redirect_uri=https%3A%2F%2Feeclass.nthu.edu.tw%2Fservice%2Foauth%2F&scope=lmsid+userid&state=&ui_locales=zh-TW")

# Find the username and password fields
username_field = driver.find_element(By.NAME, "id")
password_field = driver.find_element(By.NAME, "password")
captcha_field = driver.find_element(By.NAME, "captcha")



# Type the username and password
username_field.send_keys(account)
password_field.send_keys(password)
captcha = input("請輸入驗證碼: ")
captcha_field.send_keys(captcha)

# Find the login button and click it
login_button = driver.find_element(By.NAME, 'action')  # Replace with the actual selector
login_button.click()

code_element = driver.find_element(By.XPATH, f"//div[text()='代碼: {class_code}']")

# 找到該代碼元素的上層元素（可能需要根據具體的 HTML 結構進行調整）
parent_element = code_element.find_element(By.XPATH, './ancestor::li')

# 在這個上層元素中找到課程的超連結並點擊
course_link = parent_element.find_element(By.XPATH, ".//a[contains(@href, '/course/')]")
href_value = course_link.get_attribute("href")
course_id = re.search(r'/course/(\d+)', href_value).group(1)
driver.get(f"https://eeclass.nthu.edu.tw/course/homeworkList/{course_id}")
# course_link.click()

hw1_element = driver.find_element(By.XPATH, f"//a[@title='{homework_name}']")

# 點擊該連結
# hw1_element.click()

# 獲取超連結的 href 屬性值
href_value = hw1_element.get_attribute("href")

# 使用正則表達式從 href 屬性值中抓取數字
hw1_number = re.search(r'/homework/(\d+)', href_value).group(1)
driver.get(f"https://eeclass.nthu.edu.tw/homework/submitList/{hw1_number}")
print(f"The number is: {hw1_number}")

fs_image_elements = driver.find_elements(By.XPATH, "//*[@id='submitList_table']")


# 找到所有 class 是 "sm-text-overflow" 的 divs
div_elements = fs_image_elements[0].find_elements(By.XPATH, "//div[@class='sm-text-overflow']")
div_id = fs_image_elements[0].find_elements(By.XPATH, "//div[@class='fs-hint']")
# 用來儲存所有的 href
hrefs = []
texts = []
for div in div_elements:
    # 找到 div 內部的 <a> 標籤
    print(div)
    try:
        a_element = div.find_element(By.XPATH, ".//a")
        # 取得 <a> 標籤的 href 屬性
        href = a_element.get_attribute('href')
        hrefs.append(href)
        print(href)
        # 找到 <a> 標籤內部的 <span> 標籤
        span_element = div.find_element(By.XPATH, ".//span[@class='text ']")

        # 取得 <span> 標籤的文字
        text = span_element.text
        texts.append(text)
    except:
        pass

id_ = []
student_id = []
for id_ in div_id:
    student_id.append(id_.get_attribute('innerHTML'))
if len(student_id) != len(texts):
    print("sth wrong")
    exit()
print(student_id)
# 輸出所有找到的 hrefs 和 文字
count = 0
for href, text in zip(hrefs, texts):
    # only need to get id student
    if count <= -1:
        count += 1
        continue
    print(f"Text: {text}, HREF: {href}, Student ID: {student_id[count]}")
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.COMMAND + 't') 
    driver.get(href)
    try:
        driver.maximize_window()
    except:
        pass
    link_to_click = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a/span[text()='批改']/..")))
    #link_to_click = driver.find_element(By.XPATH, "//a/span[text()='批改']/..")

    # 點擊這個 <a> 標籤
    link_to_click.click()
    button = driver.find_element(By.XPATH, "//button/span[text()='上傳檔案']/..")
    button.click()
    
    # 等待 input 元素出現
    wait = WebDriverWait(driver, 10)
    
    path_list = find_files_with_id(directory_path, student_id[count])
    if len(path_list) == 1:
        path = path_list[0]
    else:
        print(f"error: {text}")
        count += 1
        continue
    file_input = driver.find_element(By.ID, "homework-audit-setting-form_auditNote_uploadModal-area")
    drag_and_drop_file(file_input, path)
    time.sleep(5)
    count += 1

    

In [11]:
def check_decimal(num):
    if num % 1 == 0:  # 檢查是否有小數部分
        return False  # 沒有小數點，返回整數
    else:
        return True  # 有小數點，直接返回

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

# Initialize the Chrome driver

driver = webdriver.Chrome()
driver.set_window_size(1400,1000)

# Navigate to the login page
driver.get("https://oauth.ccxp.nthu.edu.tw/v1.1/authorize.php?response_type=code&client_id=eeclass&redirect_uri=https%3A%2F%2Feeclass.nthu.edu.tw%2Fservice%2Foauth%2F&scope=lmsid+userid&state=&ui_locales=zh-TW")

# Find the username and password fields
username_field = driver.find_element(By.NAME, "id")
password_field = driver.find_element(By.NAME, "password")
captcha_field = driver.find_element(By.NAME, "captcha")



# Type the username and password
username_field.send_keys(account)
password_field.send_keys(password)
captcha = input("請輸入驗證碼: ")
captcha_field.send_keys(captcha)

# Find the login button and click it
login_button = driver.find_element(By.NAME, 'action')  # Replace with the actual selector
login_button.click()

code_element = driver.find_element(By.XPATH, f"//div[text()='代碼: {class_code}']")

# 找到該代碼元素的上層元素（可能需要根據具體的 HTML 結構進行調整）
parent_element = code_element.find_element(By.XPATH, './ancestor::li')

# 在這個上層元素中找到課程的超連結並點擊
course_link = parent_element.find_element(By.XPATH, ".//a[contains(@href, '/course/')]")
href_value = course_link.get_attribute("href")
course_id = re.search(r'/course/(\d+)', href_value).group(1)
driver.get(f"https://eeclass.nthu.edu.tw/course/homeworkList/{course_id}")
# course_link.click()

hw1_element = driver.find_element(By.XPATH, f"//a[@title='{homework_name}']")

# 點擊該連結
# hw1_element.click()

# 獲取超連結的 href 屬性值
href_value = hw1_element.get_attribute("href")

# 使用正則表達式從 href 屬性值中抓取數字
hw1_number = re.search(r'/homework/(\d+)', href_value).group(1)
driver.get(f"https://eeclass.nthu.edu.tw/homework/submitList/{hw1_number}")
print(f"The number is: {hw1_number}")

fs_image_elements = driver.find_elements(By.XPATH, "//*[@id='submitList_table']")


# 找到所有 class 是 "sm-text-overflow" 的 divs
div_elements = fs_image_elements[0].find_elements(By.XPATH, "//div[@class='sm-text-overflow']")
div_id = fs_image_elements[0].find_elements(By.XPATH, "//div[@class='fs-hint']")
# 用來儲存所有的 href
hrefs = []
texts = []
for div in div_elements:
    # 找到 div 內部的 <a> 標籤
    a_element = div.find_element(By.XPATH, ".//a")
    
    # 取得 <a> 標籤的 href 屬性
    href = a_element.get_attribute('href')
    hrefs.append(href)
    
    # 找到 <a> 標籤內部的 <span> 標籤
    span_element = div.find_element(By.XPATH, ".//span[@class='text ']")
    
    # 取得 <span> 標籤的文字
    text = span_element.text
    texts.append(text)

id_ = []
student_id = []
for id_ in div_id:
    student_id.append(id_.get_attribute('innerHTML'))
if len(student_id) != len(texts):
    print("sth wrong")
    exit()

# 輸出所有找到的 hrefs 和 文字
count = 0
for href, text in zip(hrefs, texts):
    print(f"Text: {text}, HREF: {href}, Student ID: {student_id[count]}")
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.COMMAND + 't') 
    driver.get(href)
    try:
        driver.maximize_window()
    except:
        pass
    link_to_click = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a/span[text()='批改']/..")))
    #link_to_click = driver.find_element(By.XPATH, "//a/span[text()='批改']/..")

    # 點擊這個 <a> 標籤
    link_to_click.click()
    score_field = driver.find_element(By.NAME, "auditScore")
    score = get_value_based_on_id(score_df, "NAME+id", "Score", student_id[count])
    print(score)
    if len(score) == 1:
        if check_decimal(score[0]):
            print("no need change")
            print(score[0])
            score_field.send_keys((score[0]))
        else:
            score_field.send_keys(int(score[0]))
        submit_button = driver.find_element(By.XPATH, "//button[contains(@class, 'submitAudit') and .//span[text()='送出']]")
        submit_button.click()
        count += 1
        time.sleep(5)
    else:
        print("error score")
        count += 1
    